# K-nearest neighbor (KNN) for binary classification

Implementation in knn.py

#### Some notes

In this task, we will use three distance functions: (we removed the vector symbol for simplicity)

- Euclidean distance:  $$d(x, y) = \sqrt{\langle x - y, x - y \rangle}$$
- Inner product distance: $$d(x, y ) = \langle x, y \rangle$$
- Gaussian kernel distance: 
    $$d(x, y ) = \exp({−\frac 12 \sqrt{\langle x - y, x - y \rangle}}) $$


F1-score is a important metric for binary classification, as sometimes the accuracy metric has the false positive (a good example is in MLAPP book 2.2.3.1 “Example: medical diagnosis”, Page 29).

### Distance Functions

functions in *utils.py*    
    - f1_score
    - euclidean_distance
    - inner_product_distance
    - gaussian_kernel_distance

In [3]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from k_nearest_neighbor.knn import KNN
from k_nearest_neighbor.utils import euclidean_distance, gaussian_kernel_distance, inner_product_distance
from k_nearest_neighbor.utils import f1_score

distance_funcs = {
    'euclidean': euclidean_distance,
    'gaussian': gaussian_kernel_distance,
    'inner_prod': inner_product_distance,
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Data processing 

Do the following steps:

- Load data (features and values) from function generate data cancer
- Check that there are 569 data samples and each sample have a feature vector of length 30.
- Split the whole data set into three parts:
     - the train set contains first 400 samples (0th - 399th samples),
     - the validation set contains the next 60 samples (400th - 459th samples),
     - the test set contains the rest 109 samples (460th - 568th samples).

In [4]:
from k_nearest_neighbor.data import generate_data_cancer
features, labels = generate_data_cancer()

train_features, train_labels = features[:400], labels[:400]
valid_features, valid_labels = features[400:460], labels[400:460]
test_features, test_labels = features[460:], labels[460:]

assert len(train_features) == len(train_labels) == 400
assert len(valid_features) == len(valid_labels) == 60
assert len(test_features) == len(test_labels) == 109

#### Model selection 
In kNN model, the parameter k is a hyper-parameter. In this task, we only search k among {1, 3, 10, 20, 50}.

In [5]:
for name, func in distance_funcs.items():
    best_f1_score, best_k = -1, 0
    for k in [1, 3, 10, 20, 50]:
        model = KNN(k=k, distance_function=func)
        model.train(train_features, train_labels)
        train_f1_score = f1_score(
            train_labels, model.predict(train_features))

        valid_f1_score = f1_score(
            valid_labels, model.predict(valid_features))
        print('[Model Selection] {name}\tk: {k:d}\t'.format(name=name, k=k) + 
              'train: {train_f1_score:.5f}\t'.format(train_f1_score=train_f1_score) +
              'valid: {valid_f1_score:.5f}'.format(valid_f1_score=valid_f1_score))

        if valid_f1_score > best_f1_score:
            best_f1_score, best_k = valid_f1_score, k

    model = KNN(k=best_k, distance_function=func)
    model.train(train_features + valid_features,
                train_labels + valid_labels)
    test_f1_score = f1_score(test_labels, model.predict(test_features))
    print()
    print('[Model Selection] {name}\tbest_k: {best_k:d}\t'.format(name=name, best_k=best_k) +
          'test f1 score: {test_f1_score:.5f}'.format(test_f1_score=test_f1_score))
    print()

[Model Selection] euclidean	k: 1	train: 1.00000	valid: 0.96774
[Model Selection] euclidean	k: 3	train: 0.95879	valid: 0.97872
[Model Selection] euclidean	k: 10	train: 0.95259	valid: 0.97872
[Model Selection] euclidean	k: 20	train: 0.94444	valid: 0.97872
[Model Selection] euclidean	k: 50	train: 0.92178	valid: 0.96774

[Model Selection] euclidean	best_k: 3	test f1 score: 0.95000

[Model Selection] gaussian	k: 1	train: 1.00000	valid: 0.96774
[Model Selection] gaussian	k: 3	train: 0.95879	valid: 0.97872
[Model Selection] gaussian	k: 10	train: 0.95259	valid: 0.97872
[Model Selection] gaussian	k: 20	train: 0.94444	valid: 0.97872
[Model Selection] gaussian	k: 50	train: 0.92178	valid: 0.96774

[Model Selection] gaussian	best_k: 3	test f1 score: 0.95000

[Model Selection] inner_prod	k: 1	train: 0.72408	valid: 0.87850
[Model Selection] inner_prod	k: 3	train: 0.72408	valid: 0.87850
[Model Selection] inner_prod	k: 10	train: 0.72408	valid: 0.87850
[Model Selection] inner_prod	k: 20	train: 0.72408	v

### Data transformation

We are going to add one more step (data transformation) in the data processing part and see how it works. 
Sometimes, normalization plays an important role to make a machine learning model work (check term “Feature scaling” in wiki).

Here, we take two different data transformation approaches.

#### Normalizing the feature vector 

This one is simple but some times may work well. Given a feature vector $x$, the normalized feature vector is given by 

$$ x' = \frac x {\sqrt{\langle x, x \rangle}} $$
If a vector is a all-zero vector, we let the normalized vector also be a all-zero vector.


#### Min-max scaling the feature matrix

The above normalization is data independent, that is to say, the output of the normalization function doesn’t depend on the rest training data. However, sometimes it would be helpful to do data dependent normalization. One thing to note is that, when doing data dependent normalization, we can only use training data, as the test data is assumed to be unknown during training (at least for most classification tasks).

The min-max scaling works as follows: after min-max scaling, all values of training data’s feature vectors are in the given range.
Note that this doesn’t mean the values of the validation/test data’s fea- tures are all in that range, because the validation/test data may have dif- ferent distribution as the training data.

The functions in *utils.py*    
    - normalize
    - min_max_scale

In [6]:
from k_nearest_neighbor.utils import NormalizationScaler, MinMaxScaler

scaling_functions = {
    'min_max_scale': MinMaxScaler,
    'normalize': NormalizationScaler,
}

#### Model selection

Repeat the model selection part

In [7]:
for scaling_name, scaling_class in scaling_functions.items():
    for name, func in distance_funcs.items():
        scaler = scaling_class()
        train_features_scaled = scaler(train_features)
        valid_features_scaled = scaler(valid_features)

        best_f1_score, best_k = 0, 1
        for k in [1, 3, 10, 20, 50]:
            model = KNN(k=k, distance_function=func)
            model.train(train_features_scaled, train_labels)
            train_f1_score = f1_score(
                train_labels, model.predict(train_features_scaled))
            
            valid_f1_score = f1_score(
                valid_labels, model.predict(valid_features_scaled))
            print('[Model Selection] {name}\t{scaling_name}\tk: {k:d}\t'.format(name=name, scaling_name=scaling_name, k=k) +
                  'train: {train_f1_score:.5f}\t'.format(train_f1_score=train_f1_score) + 
                  'valid: {valid_f1_score:.5f}'.format(valid_f1_score=valid_f1_score))


            if valid_f1_score > best_f1_score:
                best_f1_score, best_k = valid_f1_score, k
    

        # now change it to new scaler, since the training set changes
        scaler = scaling_class()
        combined_features_scaled = scaler(train_features + valid_features)
        test_features_scaled = scaler(test_features)

        model = KNN(k=best_k, distance_function=func)
        model.train(combined_features_scaled, train_labels + valid_labels)
        test_f1_score = f1_score(test_labels, model.predict(test_features_scaled))
        print()
        print('[Model Selection] {name}\t{scaling_name}\t'.format(name=name, scaling_name=scaling_name) +
              'best_k: {best_k:d}\ttest: {test_f1_score:.5f}'.format(best_k=best_k, test_f1_score=test_f1_score))
        print()

[Model Selection] euclidean	min_max_scale	k: 1	train: 1.00000	valid: 0.95652
[Model Selection] euclidean	min_max_scale	k: 3	train: 0.98253	valid: 0.98947
[Model Selection] euclidean	min_max_scale	k: 10	train: 0.97155	valid: 0.97872
[Model Selection] euclidean	min_max_scale	k: 20	train: 0.97826	valid: 0.97872
[Model Selection] euclidean	min_max_scale	k: 50	train: 0.95726	valid: 0.98947

[Model Selection] euclidean	min_max_scale	best_k: 3	test: 0.96296

[Model Selection] gaussian	min_max_scale	k: 1	train: 1.00000	valid: 0.95652
[Model Selection] gaussian	min_max_scale	k: 3	train: 0.98253	valid: 0.98947
[Model Selection] gaussian	min_max_scale	k: 10	train: 0.97155	valid: 0.97872
[Model Selection] gaussian	min_max_scale	k: 20	train: 0.97826	valid: 0.97872
[Model Selection] gaussian	min_max_scale	k: 50	train: 0.95726	valid: 0.98947

[Model Selection] gaussian	min_max_scale	best_k: 3	test: 0.96296

[Model Selection] inner_prod	min_max_scale	k: 1	train: 0.72408	valid: 0.87850
[Model Selection